In [69]:
import numpy as np
import itertools
from collections import Counter
from langdetect import detect
import re

In [35]:
fo = open('q_corpus_5500_clean.txt', 'w')
with open('q_corpus_5500.txt', 'r') as f:
    for l in f:
        fo.write(l.split(' ',1)[1])

In [36]:
from collections import Counter, deque
from nltk.tokenize import regexp_tokenize
import pandas as pd

def grouper(iterable, length=2):
    i = iter(iterable)
    q = deque(map(next, [i] * length))
    while True:
        yield tuple(q)
        try:
            q.append(next(i))
            q.popleft()
        except StopIteration:
            break

def tokenize(text):
    return [word.lower() for word in regexp_tokenize(text, r'\w+')]

def follow_probability(word1, word2, vec):
    subvec = vec.loc[word1]
    try:
        ct = subvec.loc[word2]
    except:
        ct = 0
    return float(ct) / (subvec.sum() or 1)

with open('q_corpus_5500_clean.txt', 'r') as f:
    text = f.read()
tokens = tokenize(text)
markov = list(grouper(tokens))
vec = pd.Series(Counter(markov))

In [90]:
follow_probability('are', 'any', vec)

0.0

In [104]:
f = open('word_dump.txt', 'r')
words = [w.strip() for w in f.readlines()]
word_keys = [w for w in words if len(w) > 3]
f.close()

(77722, 76754)

In [ ]:
question_starts = ['who', 'what', 'where', 'when', 'why', 'is', 'are', 'whom', \
                   'how', 'if', 'would', 'were', 'art', 'has', 'which', \
                   'had', 'will', 'can', 'could', 'have']
follow_words = []
for qs in question_starts:
    print(qs)
    follow_words.append([w for w in word_keys if follow_probability(qs, w, vec) > 0])

In [70]:
def vigenere_decrypt(cipher, key):
    plain = ""
    k = 0
    for c in cipher:
        plain+=(numl(rollback_c(c, key[k])))
        k = (k + 1) % len(key)
    return plain

def vigenere_key(cipher, plain):
    key = ""
    for i in range(len(cipher)):
        key+=numl(ltr_diff(plain[i], cipher[i]))
    return key

In [71]:
ltrn = lambda l : ord(l) - ord('a')
numl = lambda n : chr(n + ord('a'))
rollback_n = lambda x,r : (ltrn(x) - r) % 26
rollback_c = lambda x,r : (ltrn(x) - ltrn(r)) % 26
ltr_diff = lambda x,y : (ord(y) - ord(x)) % 26

In [72]:
follow_frags = []
for i in range(len(question_starts)):
    follow_frags.append(list(set([fw[:6 - len(question_starts[0])] for fw in follow_words[i]])))

In [82]:
plain_frags = []
for i in range(len(question_starts)):
    plain_frags.append([question_starts[i] + ff for ff in follow_frags[i] if len(question_starts[i]+ff) == 6])
    

In [94]:
vg = "euzylleuitlvxtouwnfyoajcftzmzcftj"
possible_plains = []
for i in range(len(question_starts)):
    possible_plains+=([vigenere_decrypt(vg, vigenere_key(vg[:6],pf)) for pf in plain_frags[i]])

In [105]:
better_plains  = []
for w in word_keys:
    better_plains+=[p for p in possible_plains if p.endswith(w)]

In [106]:
better_plains

['howyouhoftoeanluzwislamlinwmcling',
 'howlouhofgoeanlhzwislnmlinwzcling',
 'howwouhofroeanlszwislymlinwkcling',
 'howcomhofxowanlyzoislemdinwqcding',
 'howofthofjfdanlkqvislqdkinwctking',
 'howhothofcodanldzvisljmkinwvcking',
 'howmuchofhumanlifeislostinwaiting',
 'howeffhofzfpanlaqhislgdwinwstwing']

In [108]:
vigenere_key('howmuc', vg[:6])

'duxojr'